# CS 5228 - Team pandas


A0105650R - Wang Gejing 

A0198889R - Chen Ningshuang 

A0210996X - Zhang Hao 

## Import libraries

In [1]:
# All Imports
from utils import *
import pandas as pd
import locale
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# model evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.ensemble import *

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_ALL,'')
pd.set_option('display.max_columns', None)

## Import training data

In [2]:
# Process Training Data
drop_columns = []

le = generate_labels()
base_dropna = get_data(le=le,type='train', dropna=True, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)
base_fillna = get_data(le=le,type='train', dropna=False, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)
feature_dropna = get_data(le=le,type='train', dropna=True, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)
feature_fillna = get_data(le=le,type='train', dropna=False, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)

## Training dataset Information

In [3]:
base_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49808 entries, 0 to 49999
Data columns (total 28 columns):
Name                 49808 non-null int64
City                 49808 non-null int64
State                49808 non-null int64
Zip                  49808 non-null int32
Bank                 49808 non-null int64
BankState            49808 non-null int64
NAICS                49808 non-null int32
ApprovalDate         49808 non-null int64
ApprovalFY           49808 non-null int16
Term                 49808 non-null int64
NoEmp                49808 non-null int64
CreateJob            49808 non-null int64
RetainedJob          49808 non-null int64
FranchiseCode        49808 non-null int32
DisbursementDate     49808 non-null int64
DisbursementGross    49808 non-null float32
GrAppv               49808 non-null float32
SBA_Appv             49808 non-null float32
ChargeOff            49808 non-null int64
NewExist_1           49808 non-null uint8
NewExist_2           49808 non-null uint8
Urb

In [4]:
base_fillna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 28 columns):
Name                 50000 non-null int64
City                 50000 non-null int64
State                50000 non-null int64
Zip                  50000 non-null int32
Bank                 50000 non-null int64
BankState            50000 non-null int64
NAICS                50000 non-null int32
ApprovalDate         50000 non-null int64
ApprovalFY           50000 non-null int16
Term                 50000 non-null int64
NoEmp                50000 non-null int64
CreateJob            50000 non-null int64
RetainedJob          50000 non-null int64
FranchiseCode        50000 non-null int32
DisbursementDate     50000 non-null int64
DisbursementGross    50000 non-null float32
GrAppv               50000 non-null float32
SBA_Appv             50000 non-null float32
ChargeOff            50000 non-null int64
NewExist_1           50000 non-null uint8
NewExist_2           50000 non-null uint8
Urb

In [5]:
feature_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49808 entries, 0 to 49999
Data columns (total 34 columns):
Name                 49808 non-null int64
City                 49808 non-null int64
State                49808 non-null int64
Zip                  49808 non-null int32
Bank                 49808 non-null int64
BankState            49808 non-null int64
NAICS                49808 non-null int32
ApprovalDate         49808 non-null int64
ApprovalFY           49808 non-null int16
CreateJob            49808 non-null int64
RetainedJob          49808 non-null int64
FranchiseCode        49808 non-null int32
DisbursementDate     49808 non-null int64
DisbursementGross    49808 non-null float32
GrAppv               49808 non-null float32
SBA_Appv             49808 non-null float32
ChargeOff            49808 non-null int64
NewExist_1           49808 non-null uint8
NewExist_2           49808 non-null uint8
UrbanRural_0         49808 non-null uint8
UrbanRural_1         49808 non-null uint8
Urb

In [6]:
feature_fillna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 34 columns):
Name                 50000 non-null int64
City                 50000 non-null int64
State                50000 non-null int64
Zip                  50000 non-null int32
Bank                 50000 non-null int64
BankState            50000 non-null int64
NAICS                50000 non-null int32
ApprovalDate         50000 non-null int64
ApprovalFY           50000 non-null int16
CreateJob            50000 non-null int64
RetainedJob          50000 non-null int64
FranchiseCode        50000 non-null int32
DisbursementDate     50000 non-null int64
DisbursementGross    50000 non-null float32
GrAppv               50000 non-null float32
SBA_Appv             50000 non-null float32
ChargeOff            50000 non-null int64
NewExist_1           50000 non-null uint8
NewExist_2           50000 non-null uint8
UrbanRural_0         50000 non-null uint8
UrbanRural_1         50000 non-null uint8
Urb

## Import test data

In [7]:
# Process Test Data
feature_test = get_data(le=le,type='test', dropna=False, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)
base_test = get_data(le=le,type='test', dropna=False, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)

## Test dataset information

In [8]:
feature_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
Name                 100000 non-null int64
City                 100000 non-null int64
State                100000 non-null int64
Zip                  100000 non-null int32
Bank                 100000 non-null int64
BankState            100000 non-null int64
NAICS                100000 non-null int32
ApprovalDate         100000 non-null int64
ApprovalFY           100000 non-null float64
CreateJob            100000 non-null int64
RetainedJob          100000 non-null int64
FranchiseCode        100000 non-null int32
DisbursementDate     100000 non-null int64
DisbursementGross    100000 non-null float32
GrAppv               100000 non-null float32
SBA_Appv             100000 non-null float32
NewExist_1           100000 non-null uint8
NewExist_2           100000 non-null uint8
UrbanRural_0         100000 non-null uint8
UrbanRural_1         100000 non-null uint8
UrbanRural_2         1

In [9]:
base_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
Name                 100000 non-null int64
City                 100000 non-null int64
State                100000 non-null int64
Zip                  100000 non-null int32
Bank                 100000 non-null int64
BankState            100000 non-null int64
NAICS                100000 non-null int32
ApprovalDate         100000 non-null int64
ApprovalFY           100000 non-null float64
Term                 100000 non-null int64
NoEmp                100000 non-null int64
CreateJob            100000 non-null int64
RetainedJob          100000 non-null int64
FranchiseCode        100000 non-null int32
DisbursementDate     100000 non-null int64
DisbursementGross    100000 non-null float32
GrAppv               100000 non-null float32
SBA_Appv             100000 non-null float32
NewExist_1           100000 non-null uint8
NewExist_2           100000 non-null uint8
UrbanRural_0         1

## Data and model selection

In [10]:
model_names = ['KNN', 'LR', 'DT', 'RF', 'GBM','Ada Boost']
base_dropna_f1 = []
base_dropna_acc = []
base_fillna_f1 = []
base_fillna_acc = []
feature_dropna_f1 = []
feature_dropna_acc = []
feature_fillna_f1 = []
feature_fillna_acc = []

def calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    f1 = round(f1_score(y_test, y_pred, average='weighted') * 100, 2)
    acc = round(accuracy_score(y_test, y_pred) * 100, 2)
    return f1, acc

    
def train_single_classifier(classifier, df_in, f1_list, acc_list):
    df_x = df_in.drop(columns='ChargeOff')
    df_y = df_in['ChargeOff']
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state=0)
    f1, acc = calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test)
    f1_list.append(f1)
    acc_list.append(acc)
    

def train_model(df_in, f1_list, acc_list):
    train_single_classifier(KNeighborsClassifier(), df_in, f1_list, acc_list)
    train_single_classifier(LogisticRegression(), df_in, f1_list, acc_list)
    train_single_classifier(DecisionTreeClassifier(), df_in, f1_list, acc_list)
    train_single_classifier(RandomForestClassifier(random_state=1234), df_in, f1_list, acc_list)
    train_single_classifier(GradientBoostingClassifier(random_state=1234), df_in, f1_list, acc_list)
    train_single_classifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234),random_state=1234), df_in, f1_list, acc_list)


    

train_model(base_dropna, base_dropna_f1, base_dropna_acc)
train_model(base_fillna, base_fillna_f1, base_fillna_acc)
train_model(feature_dropna, feature_dropna_f1, feature_dropna_acc)
train_model(feature_fillna, feature_fillna_f1, feature_fillna_acc)


In [11]:
accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'base_fillna_acc': base_fillna_acc, 'feature_dropna_acc': feature_dropna_acc, 'feature_fillna_acc': feature_fillna_acc})
# accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'feature_dropna_acc': feature_dropna_acc})
accuracy_record['acc_mean'] = accuracy_record.mean(axis=1).round(2)
accuracy_record.set_index('Model', inplace=True)
accuracy_record.loc['avg'] = accuracy_record.mean()

F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'base_fillna_f1': base_fillna_f1, 'feature_dropna_f1': feature_dropna_f1, 'feature_fillna_f1': feature_fillna_f1})
# F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'feature_dropna_f1': feature_dropna_f1})
F1_record['F1_mean'] = F1_record.mean(axis=1).round(2)
F1_record.set_index('Model', inplace=True)
F1_record.loc['avg'] = F1_record.mean()


In [12]:
accuracy_record.head(10)

,base_dropna_acc,base_fillna_acc,feature_dropna_acc,feature_fillna_acc,acc_mean
Model,,,,,
KNN,68.500000,68.460,68.500000,68.460,68.48
LR,63.760000,61.220,63.750000,61.220,62.49
DT,87.870000,87.320,71.680000,72.060,79.73
RF,90.220000,89.900,79.560000,79.160,84.71
GBM,90.100000,89.590,77.680000,77.870,83.81
Ada Boost,87.720000,87.500,71.800000,71.880,79.72
avg,81.361667,80.665,72.161667,71.775,76.49


In [13]:
F1_record.head(10)

,base_dropna_f1,base_fillna_f1,feature_dropna_f1,feature_fillna_f1,F1_mean
Model,,,,,
KNN,68.490000,68.46,68.490,68.460000,68.470000
LR,63.030000,59.95,63.010,59.950000,61.480000
DT,87.870000,87.32,71.690,72.060000,79.740000
RF,90.210000,89.89,79.540,79.130000,84.690000
GBM,90.100000,89.59,77.680,77.860000,83.810000
Ada Boost,87.720000,87.49,71.800,71.880000,79.720000
avg,81.236667,80.45,72.035,71.556667,76.318333


## Drop columns

In [14]:
# Process Training Data
drop_columns = ['CreateJob','RetainedJob','City','Name','Zip','BankState']

base_dropna = get_data(le=le,type='train', dropna=True, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)
base_fillna = get_data(le=le,type='train', dropna=False, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)
feature_dropna = get_data(le=le,type='train', dropna=True, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)
feature_fillna = get_data(le=le,type='train', dropna=False, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)

## Training dataset Information

In [15]:
base_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49808 entries, 0 to 49999
Data columns (total 22 columns):
State                49808 non-null int64
Bank                 49808 non-null int64
NAICS                49808 non-null int32
ApprovalDate         49808 non-null int64
ApprovalFY           49808 non-null int16
Term                 49808 non-null int64
NoEmp                49808 non-null int64
FranchiseCode        49808 non-null int32
DisbursementDate     49808 non-null int64
DisbursementGross    49808 non-null float32
GrAppv               49808 non-null float32
SBA_Appv             49808 non-null float32
ChargeOff            49808 non-null int64
NewExist_1           49808 non-null uint8
NewExist_2           49808 non-null uint8
UrbanRural_0         49808 non-null uint8
UrbanRural_1         49808 non-null uint8
UrbanRural_2         49808 non-null uint8
RevLineCr_N          49808 non-null uint8
RevLineCr_Y          49808 non-null uint8
LowDoc_N             49808 non-null uint8
Low

In [16]:
base_fillna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
State                50000 non-null int64
Bank                 50000 non-null int64
NAICS                50000 non-null int32
ApprovalDate         50000 non-null int64
ApprovalFY           50000 non-null int16
Term                 50000 non-null int64
NoEmp                50000 non-null int64
FranchiseCode        50000 non-null int32
DisbursementDate     50000 non-null int64
DisbursementGross    50000 non-null float32
GrAppv               50000 non-null float32
SBA_Appv             50000 non-null float32
ChargeOff            50000 non-null int64
NewExist_1           50000 non-null uint8
NewExist_2           50000 non-null uint8
UrbanRural_0         50000 non-null uint8
UrbanRural_1         50000 non-null uint8
UrbanRural_2         50000 non-null uint8
RevLineCr_N          50000 non-null uint8
RevLineCr_Y          50000 non-null uint8
LowDoc_N             50000 non-null uint8
Low

In [17]:
feature_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49808 entries, 0 to 49999
Data columns (total 28 columns):
State                49808 non-null int64
Bank                 49808 non-null int64
NAICS                49808 non-null int32
ApprovalDate         49808 non-null int64
ApprovalFY           49808 non-null int16
FranchiseCode        49808 non-null int32
DisbursementDate     49808 non-null int64
DisbursementGross    49808 non-null float32
GrAppv               49808 non-null float32
SBA_Appv             49808 non-null float32
ChargeOff            49808 non-null int64
NewExist_1           49808 non-null uint8
NewExist_2           49808 non-null uint8
UrbanRural_0         49808 non-null uint8
UrbanRural_1         49808 non-null uint8
UrbanRural_2         49808 non-null uint8
RevLineCr_N          49808 non-null uint8
RevLineCr_Y          49808 non-null uint8
LowDoc_N             49808 non-null uint8
LowDoc_Y             49808 non-null uint8
NoEmp_Micro          49808 non-null uint8
NoE

In [18]:
feature_fillna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 28 columns):
State                50000 non-null int64
Bank                 50000 non-null int64
NAICS                50000 non-null int32
ApprovalDate         50000 non-null int64
ApprovalFY           50000 non-null int16
FranchiseCode        50000 non-null int32
DisbursementDate     50000 non-null int64
DisbursementGross    50000 non-null float32
GrAppv               50000 non-null float32
SBA_Appv             50000 non-null float32
ChargeOff            50000 non-null int64
NewExist_1           50000 non-null uint8
NewExist_2           50000 non-null uint8
UrbanRural_0         50000 non-null uint8
UrbanRural_1         50000 non-null uint8
UrbanRural_2         50000 non-null uint8
RevLineCr_N          50000 non-null uint8
RevLineCr_Y          50000 non-null uint8
LowDoc_N             50000 non-null uint8
LowDoc_Y             50000 non-null uint8
NoEmp_Micro          50000 non-null uint8
NoE

## Import test data

In [19]:
# Process Test Data
feature_test = get_data(le=le,type='test', dropna=False, get_dummy=True, feature_split=True, values_only=True,drop_columns=drop_columns)
base_test = get_data(le=le,type='test', dropna=False, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)

## Test dataset information

In [20]:
feature_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
State                100000 non-null int64
Bank                 100000 non-null int64
NAICS                100000 non-null int32
ApprovalDate         100000 non-null int64
ApprovalFY           100000 non-null float64
FranchiseCode        100000 non-null int32
DisbursementDate     100000 non-null int64
DisbursementGross    100000 non-null float32
GrAppv               100000 non-null float32
SBA_Appv             100000 non-null float32
NewExist_1           100000 non-null uint8
NewExist_2           100000 non-null uint8
UrbanRural_0         100000 non-null uint8
UrbanRural_1         100000 non-null uint8
UrbanRural_2         100000 non-null uint8
RevLineCr_N          100000 non-null uint8
RevLineCr_Y          100000 non-null uint8
LowDoc_N             100000 non-null uint8
LowDoc_Y             100000 non-null uint8
NoEmp_Micro          100000 non-null uint8
NoEmp_Small          1

In [21]:
base_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
State                100000 non-null int64
Bank                 100000 non-null int64
NAICS                100000 non-null int32
ApprovalDate         100000 non-null int64
ApprovalFY           100000 non-null float64
Term                 100000 non-null int64
NoEmp                100000 non-null int64
FranchiseCode        100000 non-null int32
DisbursementDate     100000 non-null int64
DisbursementGross    100000 non-null float32
GrAppv               100000 non-null float32
SBA_Appv             100000 non-null float32
NewExist_1           100000 non-null uint8
NewExist_2           100000 non-null uint8
UrbanRural_0         100000 non-null uint8
UrbanRural_1         100000 non-null uint8
UrbanRural_2         100000 non-null uint8
RevLineCr_N          100000 non-null uint8
RevLineCr_Y          100000 non-null uint8
LowDoc_N             100000 non-null uint8
LowDoc_Y             1

## Data and model selection

In [22]:
model_names = ['KNN', 'LR', 'DT', 'RF', 'GBM','Ada Boost']
base_dropna_f1 = []
base_dropna_acc = []
base_fillna_f1 = []
base_fillna_acc = []
feature_dropna_f1 = []
feature_dropna_acc = []
feature_fillna_f1 = []
feature_fillna_acc = []

def calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    f1 = round(f1_score(y_test, y_pred, average='weighted') * 100, 2)
    acc = round(accuracy_score(y_test, y_pred) * 100, 2)
    return f1, acc

    
def train_single_classifier(classifier, df_in, f1_list, acc_list):
    df_x = df_in.drop(columns='ChargeOff')
    df_y = df_in['ChargeOff']
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state=0)
    f1, acc = calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test)
    f1_list.append(f1)
    acc_list.append(acc)
    

def train_model(df_in, f1_list, acc_list):
    train_single_classifier(KNeighborsClassifier(), df_in, f1_list, acc_list)
    train_single_classifier(LogisticRegression(), df_in, f1_list, acc_list)
    train_single_classifier(DecisionTreeClassifier(), df_in, f1_list, acc_list)
    train_single_classifier(RandomForestClassifier(random_state=1234), df_in, f1_list, acc_list)
    train_single_classifier(GradientBoostingClassifier(random_state=1234), df_in, f1_list, acc_list)
    train_single_classifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234),random_state=1234), df_in, f1_list, acc_list)

    

train_model(base_dropna, base_dropna_f1, base_dropna_acc)
train_model(base_fillna, base_fillna_f1, base_fillna_acc)
train_model(feature_dropna, feature_dropna_f1, feature_dropna_acc)
train_model(feature_fillna, feature_fillna_f1, feature_fillna_acc)


In [23]:
accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'base_fillna_acc': base_fillna_acc, 'feature_dropna_acc': feature_dropna_acc, 'feature_fillna_acc': feature_fillna_acc})
# accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'feature_dropna_acc': feature_dropna_acc})
accuracy_record['acc_mean'] = accuracy_record.mean(axis=1).round(2)
accuracy_record.set_index('Model', inplace=True)
accuracy_record.loc['avg'] = accuracy_record.mean()

F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'base_fillna_f1': base_fillna_f1, 'feature_dropna_f1': feature_dropna_f1, 'feature_fillna_f1': feature_fillna_f1})
# F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'feature_dropna_f1': feature_dropna_f1})
F1_record['F1_mean'] = F1_record.mean(axis=1).round(2)
F1_record.set_index('Model', inplace=True)
F1_record.loc['avg'] = F1_record.mean()



In [24]:
accuracy_record.head(10)

,base_dropna_acc,base_fillna_acc,feature_dropna_acc,feature_fillna_acc,acc_mean
Model,,,,,
KNN,68.360000,68.420,68.360,68.420000,68.390000
LR,63.280000,61.180,63.280,61.180000,62.230000
DT,88.170000,87.820,71.200,71.320000,79.630000
RF,89.760000,89.740,78.320,78.100000,83.980000
GBM,90.090000,89.830,77.720,77.420000,83.760000
Ada Boost,88.220000,87.780,71.050,71.410000,79.620000
avg,81.313333,80.795,71.655,71.308333,76.268333


In [25]:
F1_record.head(10)

,base_dropna_f1,base_fillna_f1,feature_dropna_f1,feature_fillna_f1,F1_mean
Model,,,,,
KNN,68.360000,68.420000,68.360000,68.420000,68.390000
LR,62.380000,59.900000,62.380000,59.900000,61.140000
DT,88.170000,87.820000,71.200000,71.320000,79.630000
RF,89.760000,89.740000,78.300000,78.060000,83.960000
GBM,90.090000,89.830000,77.720000,77.410000,83.760000
Ada Boost,88.220000,87.780000,71.050000,71.410000,79.620000
avg,81.163333,80.581667,71.501667,71.086667,76.083333
